In [3]:
import networkx as nx
from config import CONFIG
from metro_dataset import MetroDataset
from config import Line
from pytorch_lightning import Trainer
from metro_model import MetroModel
from networkx.drawing.layout import *
import numpy as np
import torch
torch.manual_seed(0)

# Dataset

In [4]:
dataset = MetroDataset(CONFIG.lines, init_nb=20)
num_nodes = dataset.cg.num_nodes()
dataset.cg.altair_graph(dataset.cg.graph, 10)

alt.LayerChart(...)

# Model

In [5]:
A_init = torch.tensor(dataset.cg.adjacency_matrix(), requires_grad=True).float()

In [6]:
model = MetroModel(embedding_size=8, num_nodes=num_nodes, neighbor_nb=2, input_size=1, gsl_mode="embedding", lr=1e-3) # , 

# Training

In [12]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=8)
trainer = Trainer(max_epochs=100)
trainer.fit(model, train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                   | Type      | Params
-----------------------------------------------------
0 | node_embeddings_start  | Embedding | 320   
1 | node_embeddings_target | Embedding | 320   
2 | _linear1               | Linear    | 72    
3 | _linear2               | Linear    | 72    
4 | graph_layer            | GraphConv | 3     
5 | linear                 | Linear    | 1.6 K 
6 | softmax                | Softmax   | 0     
-----------------------------------------------------
4.0 K     Trainable params
0         Non-trainable params
4.0 K     Total params
0.016     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 248/248 [00:01<00:00, 148.55it/s, loss=0.00711, v_num=51]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 248/248 [00:01<00:00, 147.84it/s, loss=0.00711, v_num=51]


# Evaluation

In [18]:
A = model.graph_learning()

dim=1
values, indices = A.topk(k=2, dim=dim)
mask = torch.zeros_like(A)
mask.scatter_(dim, indices, values.fill_(1))
A = A*mask

A = A.detach().numpy()
A = np.array(A)

In [20]:
B = A * (A > 0.1)

In [21]:
learned_graph = nx.from_numpy_array(A, nx.DiGraph)
dataset.cg.altair_graph(learned_graph, 10)

alt.LayerChart(...)

In [11]:
# TODO: Essayer avec les embeddings
# TODO: Essayer avec un graph bruité en entrée
# TODO: Essayer avec des data bruitées
# TODO: Essayer avec différents réseaux (de métro)
# TODO: Essayer avec le graphe contracté 